In [ ]:
from copy import copy
from random import randint, random

import numpy as np
from numpy import mean

UKRYTE_HASLO = "4729210302"
L_CHROMOSOMOW = 10


# Zadanie
Chcemy uzyskac haslo do systemu. Wiemy ze haslo sklada sie z 10 cyfr, ale jakie to cyfry?
Mozemy sprobowac znalezc to haslo w sposob losowy, np. wypisujac kolejno 0123456789, 0123456790, ...
Ale mozemy wykorzystac tez algorytmy genetyczne.

### Algorytm genetyczny
Algorytm genetyczny opiera sie na ulepszaniu populacji. Populacja sklada sie z osobnikow, a osobnik ma swoj genom.
W kazdym zastosowaniu nalezy zdefiniowac wiele rzeczy. Zacznijmy po kolei.

## Populacja
Niech pojedynczym osobnikiem bedzie potencjalne haslo. Zatem jeden osobnik to na przyklad 0123456789.
Kazdy osobnik sklada sie z genow. Tutaj geny to sa cyfry. Zatem kazdy osobnik ma 10 genow, ktore sa cyframi.
Czy kolejnosc genow ma znaczenie?
Tutaj ma, przeciez osobnicy zlozeni dokladnie z tych samych genow 0123456789 i 0123456798 reprezentuja rozne liczby.
Ale nie zawsze kolejnosc musi miec znaczenie.

Podczas inicjalizacji populacji powinnismy stworzyc kilka/kilkanascie/dziesiat osobnikow o roznym genomie.
Zacznijmy od 6 poczatkowych losowych osobnikow.

In [ ]:
# tworzenie jednego osobnika (jednego hasla)
def tworz_losowego_osobnika(l_chromosomow=10):
    if l_chromosomow < 1:
        print("Liczba chromosomow powinna byc wieksza od 0.")
    osobnik = ""
    for chromosom in range(l_chromosomow):
        osobnik = osobnik + str(randint(0, 9))
    return osobnik

osobnik = tworz_losowego_osobnika(2)
print(osobnik)

In [ ]:
# tworzenie calej populacji hasel (wiele osobnikow)
def tworz_populacje(wielkosc=10, l_chromosomow=10):
    return [
        tworz_losowego_osobnika(l_chromosomow) for i in range(wielkosc)
    ]


populacja = tworz_populacje(wielkosc=6, l_chromosomow=2)
print(populacja)

## Selekcja
Mamy jakies losowe hasla. Ale po co nam losowe hasla? Musimy umiec oceniac ich przydatnosc.
Tak sie sklada, ze system podczas logowania mowi nam ile cyfr hasla jest poprawnych.


In [ ]:
# funkcja, ktora podaje ile poprawnych cyfr ma haslo przedstawione przez danego osobnika
def podaj_liczbe_poprawnych_liter(osobnik):
    wynik = 0
    for chromosom_osobnika, poprawny_chromosom in zip(osobnik, UKRYTE_HASLO):
        if chromosom_osobnika == poprawny_chromosom:
            wynik = wynik + 1
    return wynik

print("Liczba poprawnych cyfr 23: ", podaj_liczbe_poprawnych_liter("23"))
print("Liczba poprawnych cyfr 77: ", podaj_liczbe_poprawnych_liter("77"))
print("Liczba poprawnych cyfr 47: ", podaj_liczbe_poprawnych_liter("47"))
print("Liczba poprawnych cyfr 74: ", podaj_liczbe_poprawnych_liter("74"))

In [ ]:
# robimy selekcje z calej populacji - wybieramy osobnikow blizszych poprawnemu haslu
def selekcja_populacji(populacja, wielkosc_po_selekcji=10):
    wyniki = np.array([podaj_liczbe_poprawnych_liter(osobnik) for osobnik in populacja])
    indeksy_wynikow = np.argsort(wyniki)[::-1]
    return list(np.array(populacja)[indeksy_wynikow[:wielkosc_po_selekcji]])


populacja_po_selekcji = selekcja_populacji(populacja, 3)
# poprawne haslo zaczyna sie 47 - mozemy zobaczyc, 
# ze pierwsze kilka hasel ma albo 4 na pierwszym miejscu albo 7 na drugim miejscu
print(populacja_po_selekcji)

## Mutacja
W jakis sposob powinnismy zaczac sie zblizac do poprawnego hasla. Mozemy to robic poprzez zmiany w obecnym hasle.
Dlaczego to moze dzialac? Poniewaz robimy selekcje i wybieramy te hasla ktore sa blizej poprawnego, zatem edytujemy hasla blizsze poprawnemu.

In [ ]:
# mutowanie jednego osobnika
def mutacja_osobnika(osobnik):
    osobnik = [gen for gen in osobnik]
    osobnik[randint(0, len(osobnik)-1)] = str(randint(0, 9))
    return ''.join(osobnik)

print("Zmutowany osobnik 41: ", mutacja_osobnika('41'))
print("Zmutowany osobnik 41: ", mutacja_osobnika('41'))
print("Zmutowany osobnik 41: ", mutacja_osobnika('41'))

In [ ]:
# mutowanie calej populacji
def mutacja_populacji(populacja, pstwo_mutacji=0.67):
    zmutowani_osobnicy = [
        mutacja_osobnika(osobnik) for osobnik in populacja if random() < pstwo_mutacji
    ]
    return populacja + zmutowani_osobnicy


# Po zmutowaniu populacja jest wieksza o okolo 2 osobnikow. Dlaczego? Który osobnik powstal z ktorego?
zmutowana_populacja = mutacja_populacji(populacja_po_selekcji)
print(zmutowana_populacja)
print("Stara populacja", zmutowana_populacja[:3])
print("Nowa populacja", zmutowana_populacja[3:])

## Pierwsze sprawdzenie
Sprawdzmy jak dziala nasz algorytm w tym momencie.
Uruchomimy go na 100 generacjach.

In [ ]:
L_GENERACJI = 100
PSTWO_MUTACJI = 0.4

populacja = tworz_populacje(wielkosc=20, l_chromosomow=L_CHROMOSOMOW)
for generacja in range(L_GENERACJI):
    populacja = selekcja_populacji(populacja, 10)
    print(f"Generacja {generacja+1}, najlepszy wynik {podaj_liczbe_poprawnych_liter(populacja[0])} dla hasla {populacja[0]}.")
    populacja = mutacja_populacji(populacja, PSTWO_MUTACJI)


### WOW! Udalo sie znalezc haslo! Ile prób zgadniecia hasla mielismy? Jak to obliczyc?


## Krzyzowanie
Ok, ale mamy jeszcze jednego agenta w kieszeni. Otoz w biologii osobniki sie rozmnazaja.
Podczas rozmnazania potomkowie otrzymuja geny od obu rodzicow, czyli sa usrednieniem rodzicow.
Moze sie okazac, ze od obu rodzicow dostana geny bardziej przydatne lub od obu mniej przydatne albo mieszanke.
U nas rodzicami sa hasla i potomkami tez. Jak zrobic krzyzowanie 2 hasel?
Wezmy pierwsza polowe hasla od pierwszego rodzica, a druga polowe od drugiego rodzica.

In [ ]:
# krzyzowanie 2 rodzicow, by otrzymac nowego osobnika
def krzyzuj_rodzicow(rodzic1, rodzic2):
    indeks = int(len(rodzic1)/2)
    return rodzic1[:indeks] + rodzic2[indeks:]

rodzic1 = populacja_po_selekcji[0]
rodzic2 = populacja_po_selekcji[1]
print("Rodzic 1", rodzic1)
print("Rodzic 2", rodzic2)

krzyzowanie1 = krzyzuj_rodzicow(rodzic1, rodzic2)
krzyzowanie2 = krzyzuj_rodzicow(rodzic2, rodzic1)
print("Rodzic 1 skrzyzowany z rodzicem 2", krzyzowanie1)
print("Rodzic 2 skrzyzowany z rodzicem 1", krzyzowanie2)

In [ ]:
def _szukaj_drugiego_rodzica(populacja, i):
    losowy_index = randint(0, len(populacja) - 2)
    if losowy_index >= i:
        losowy_index += 1
    return populacja[losowy_index]

# krzyzowanie populacji
def krzyzowanie_populacji(populacja, pstwo_krzyzowania=0.67):
    krzyzowani_osobnicy = [
        krzyzuj_rodzicow(osobnik, _szukaj_drugiego_rodzica(populacja, i))
        for i, osobnik in enumerate(populacja)
        if random() < pstwo_krzyzowania
    ]
    return populacja + krzyzowani_osobnicy

skrzyzowana_populacja = krzyzowanie_populacji(populacja_po_selekcji)
print(skrzyzowana_populacja)
print("Stara populacja", skrzyzowana_populacja[:3])
print("Nowa populacja", skrzyzowana_populacja[3:])

## Druga proba
Sprawdzmy jak teraz dziala nasz algorytm.
Uruchomimy go na 100 generacjach, ale z krzyzowaniem.

In [ ]:
LICZBA_PROB = 1000

def srednia_liczba_generacji_do_wyniku(pstwo_mutacji, pstwo_krzyzowania):
    generacje = []
    for proba in range(LICZBA_PROB):
        wynik = 0
        generacja = 0
        populacja = tworz_populacje(wielkosc=20, l_chromosomow=L_CHROMOSOMOW)
        while wynik<10:
            generacja += 1
            populacja = selekcja_populacji(populacja, 10)
            wynik = podaj_liczbe_poprawnych_liter(populacja[0])
            populacja = mutacja_populacji(populacja, pstwo_mutacji)
            populacja = krzyzowanie_populacji(populacja, pstwo_krzyzowania)
        generacje.append(generacja)
    return mean(generacje)


In [ ]:
l_generacji_mutacja = srednia_liczba_generacji_do_wyniku(pstwo_mutacji=0.4, pstwo_krzyzowania=0)
print(f"Srednia liczba generacji przy samej mutacji {l_generacji_mutacja}.")

l_generacji_mutacja_i_krzyzowanie = srednia_liczba_generacji_do_wyniku(pstwo_mutacji=0.2, pstwo_krzyzowania=0.2)
print(f"Srednia liczba generacji przy mutacji i krzyzowaniu {l_generacji_mutacja_i_krzyzowanie}.")


Okazuje sie ze algorytm zbiegl wolniej. Ile tym razem zostalo utworzonych hasel?
Czy potrafisz zmienic cos w tym algorytmie, by dzialal szybciej?
Czy mozna jakos zmienic krzyzowanie, mutacje?

In [ ]:
l_generacji_mutacja_i_krzyzowanie = srednia_liczba_generacji_do_wyniku(pstwo_mutacji=0.3, pstwo_krzyzowania=0.1)
print(f"Srednia liczba generacji przy mutacji i krzyzowaniu {l_generacji_mutacja_i_krzyzowanie}.")


In [ ]:
l_generacji_mutacja_i_krzyzowanie = srednia_liczba_generacji_do_wyniku(pstwo_mutacji=0.8, pstwo_krzyzowania=0.8)
print(f"Srednia liczba generacji przy mutacji i krzyzowaniu {l_generacji_mutacja_i_krzyzowanie}.")


In [ ]:
l_generacji_mutacja_i_krzyzowanie = srednia_liczba_generacji_do_wyniku(pstwo_mutacji=0.8, pstwo_krzyzowania=0.2)
print(f"Srednia liczba generacji przy mutacji i krzyzowaniu {l_generacji_mutacja_i_krzyzowanie}.")
